In [ ]:
from os import environ
environ["AP_PORT"] = "25432" # str(input('port'))
# README command uses port=25432

In [ ]:
import pickle
import json
import os
import psycopg2
import numpy as np

from spatialyze.database import Database
from spatialyze.geospatial_video import GeospatialVideo
from spatialyze.road_network import RoadNetwork
from spatialyze.video_processor.camera_config import camera_config
from spatialyze.video_processor.stages.tracking_3d.tracking_3d import Tracking3DResult
from spatialyze.world import World, _execute
from spatialyze.video_processor.cache import disable_cache
from spatialyze.video_processor.metadata_json_encoder import MetadataJSONEncoder

In [ ]:
OUTPUT_DIR = '../../data/pipeline/test-results'
VIDEO_DIR =  '../../data/pipeline/videos'
ROAD_DIR = '../../data/scenic/road-network/boston-seaport'

files = os.listdir(VIDEO_DIR)
with open(os.path.join(VIDEO_DIR, 'frames.pkl'), 'rb') as f:
    videos = pickle.load(f)
    
disable_cache()

In [ ]:
database = Database(
    psycopg2.connect(
        dbname=environ.get("AP_DB", "mobilitydb"),
        user=environ.get("AP_USER", "docker"),
        host=environ.get("AP_HOST", "localhost"),
        port=environ.get("AP_PORT", "25432"),
        password=environ.get("AP_PASSWORD", "docker"),
    )
)

In [ ]:
world = World(database)
world.addGeogConstructs(RoadNetwork('Boston-Seaport', ROAD_DIR))

In [ ]:
for video in videos.values():
    if video['filename'] not in files:
        continue
    
    videofile = os.path.join(VIDEO_DIR, video['filename'])
    camera = [camera_config(*c) for c in video['frames']]

    world.addVideo(GeospatialVideo(videofile, camera))

In [ ]:
from spatialyze.utils import F

o = world.object()
c = world.camera()
world.filter((o.type == 'car') &
    F.contained(o.trans@c.time, 'intersection')
)

In [ ]:
objects, trackings = _execute(world)

In [ ]:
import cv2
import os

def save_video_util(objects, trackings, OUTPUT_DIR, addBoundingBoxes=False):
    frame_trackings = _get_frame_objects(trackings=trackings)
    cameraIds = _get_camera_ids(objects=objects)

    for video in frame_trackings:
        cameraId = cameraIds[video]
        output_file = os.path.join(OUTPUT_DIR, cameraId + "-result.mp4")
        cap = cv2.VideoCapture(video)
        if not cap.isOpened():
            print(f"WARNING: Cannot read video file: {video}")
            continue

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        vid_writer = cv2.VideoWriter(
            output_file, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height)
        )

        print("hi")
        frame_cnt = 0
        while cap.isOpened():
            ret, frame = cap.read()

            if frame_cnt in frame_trackings[video] and ret:
                print("hi")
                if addBoundingBoxes:
                    for track in frame_trackings[video][frame_cnt]:
                        bbox_left, bbox_top, bbox_w, bbox_h = track.bbox_left, track.bbox_top, track.bbox_w, track.bbox_h
                        x1, y1 = bbox_left, bbox_top
                        x2, y2 = bbox_left + bbox_w, bbox_top - bbox_h
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 0), 2)
                
                vid_writer.write(frame)
            
            frame_cnt += 1
            if not ret:
                break
            
        cap.release()
        vid_writer.release()

"""
Indexes objects based on frame ID
"""
def _get_frame_objects(trackings):

    result = {}
    for video in trackings:
        result[video] = {}
        for frame in trackings[video]:
            for objectId in frame:
                track = frame[objectId]
                frameId = track.frame_idx
                if frameId not in result[video]:
                    result[video][frameId] = []
                result[video][frameId].append(track)

    return result

"""
Gets the cameraIds relating to each of the videos
"""
def _get_camera_ids(objects):
    result = {}
    for video in objects:
        if len(objects[video]) == 0:
            continue
    
        result[video] = objects[video][0][2]
    return result


save_video_util(objects, trackings, OUTPUT_DIR, addBoundingBoxes=True)

In [ ]:
_get_frame_objects(trackings)